In [2]:
!pip install BitVector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for BitVector: filename=BitVector-3.5.0-py3-none-any.whl size=43506 sha256=122cfbd6029a483e1391c270e989f715184640deeb6e40b8d78b4098d817eb9c
  Stored in directory: /root/.cache/pip/wheels/cf/1c/82/cb7647c0f5c92b232dcc7e775de20c3fb649f693e7607118d3
Successfully built BitVector


In [5]:
from itertools import chain
from BitVector import *
from tqdm import tqdm

# Create the Vocabulary

In [6]:
mark_twain_quotes = ["If you tell the truth, you don’t have to remember anything",
"The man who does not read has no advantage over the man who cannot read",
"Whenever you find yourself on the side of the majority, it is time to reform or pause and reflect",
"Never argue with stupid people, they will drag you down to their level and then beat you with experience",
"The two most important days in your life are the day you are born and the day you find out why",
"Kindness is the language which the deaf can hear and the blind can see",
"Continuous improvement is better than delayed perfection",
"Don’t go around saying the world owes you a living. The world owes you nothing. It was here first",
"In a good bookroom you feel in some mysterious way that you are absorbing the wisdom contained in all the books through your skin, without even opening them",
"I have never let my schooling interfere with my education"]

vocab = [quote.split(' ') for quote in mark_twain_quotes]
vocab = set(chain(*vocab))
vocab.remove('a')
vocab.remove('I')

In [7]:
vocab

{'Continuous',
 'Don’t',
 'If',
 'In',
 'It',
 'Kindness',
 'Never',
 'The',
 'Whenever',
 'absorbing',
 'advantage',
 'all',
 'and',
 'anything',
 'are',
 'argue',
 'around',
 'beat',
 'better',
 'blind',
 'bookroom',
 'books',
 'born',
 'can',
 'cannot',
 'contained',
 'day',
 'days',
 'deaf',
 'delayed',
 'does',
 'don’t',
 'down',
 'drag',
 'education',
 'even',
 'experience',
 'feel',
 'find',
 'first',
 'go',
 'good',
 'has',
 'have',
 'hear',
 'here',
 'important',
 'improvement',
 'in',
 'interfere',
 'is',
 'it',
 'language',
 'let',
 'level',
 'life',
 'living.',
 'majority,',
 'man',
 'most',
 'my',
 'mysterious',
 'never',
 'no',
 'not',
 'nothing.',
 'of',
 'on',
 'opening',
 'or',
 'out',
 'over',
 'owes',
 'pause',
 'people,',
 'perfection',
 'read',
 'reflect',
 'reform',
 'remember',
 'saying',
 'schooling',
 'see',
 'side',
 'skin,',
 'some',
 'stupid',
 'tell',
 'than',
 'that',
 'the',
 'their',
 'them',
 'then',
 'they',
 'through',
 'time',
 'to',
 'truth,',
 'two

In [8]:
PassPhrase = "Hopes and dreams of a million years"

BLOCKSIZE = 16
numbytes = BLOCKSIZE // 8

# Reduce the passphrase to a bit array of size BLOCKSIZE:
bv_iv = BitVector(bitlist = [0]*BLOCKSIZE)
for i in range(0,len(PassPhrase) // numbytes):
    textstr = PassPhrase[i*numbytes:(i+1)*numbytes]
    bv_iv ^= BitVector( textstring = textstr )

# Create a bitvector from the ciphertext hex string:
FILEIN = open("cipher.txt")
encrypted_bv = BitVector( hexstring = FILEIN.read() )
print("input cipher: ", encrypted_bv.get_hex_string_from_bitvector())

input cipher:  20352a7e36703a6930767f7276397e376528632d6b6665656f6f6424623c2d30272f3c2d3d2172396933742c7e233f687d2e32083c11385a03460d440c25


In [9]:
def decrypt(key_bv):
    # Create a bitvector for storing the decrypted plaintext bit array:
    msg_decrypted_bv = BitVector( size = 0 )

    # Carry out differential XORing of bit blocks and decryption:
    previous_decrypted_block = bv_iv
    for i in range(0, len(encrypted_bv) // BLOCKSIZE):
        bv = encrypted_bv[i*BLOCKSIZE:(i+1)*BLOCKSIZE]
        temp = bv.deep_copy()
        bv ^=  previous_decrypted_block
        previous_decrypted_block = temp
        bv ^=  key_bv
        msg_decrypted_bv += bv

    # Extract plaintext from the decrypted bitvector:
    outputtext = msg_decrypted_bv.get_text_from_bitvector()

    return outputtext

In [12]:
KEYSIZE = 2**16

result = ""
result_key = None

for key in tqdm(range(2**14, KEYSIZE)): # for all possible keys
    # Reduce the key to a bit array of size BLOCKSIZE:
    key_bv = BitVector(intVal = key, size = BLOCKSIZE)

    output = decrypt(key_bv)
    output_set = set(output.split(' '))

    intersection = output_set.intersection(vocab)

    if len(intersection) > 0:
      print("Found intersection: ", intersection)
      result = output
      result_key = key
      break

print("possible quote: ", result)
print("used key is : ", result_key)

 25%|██▍       | 12139/49152 [03:16<09:58, 61.80it/s]

Found intersection:  {'of', 'The', 'is'}
possible quote:  The secret of getting ahead is getting started. -- Mark Twain

used key is :  28523
